In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions-dataset-for-nlp/val.txt
/kaggle/input/emotions-dataset-for-nlp/test.txt
/kaggle/input/emotions-dataset-for-nlp/train.txt


In [37]:
df_test= pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/test.txt')
df_test.head()

,im feeling rather rotten so im not very ambitious right now;sadness
0,im updating my blog because i feel shitty;sadness
1,i never make her separate from me because i do...
2,i left with my bouquet of red and yellow tulip...
3,i was feeling a little vain when i did this on...
4,i cant walk into a shop anywhere where i do no...


In [38]:
df_train= pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/train.txt')
df_train.head()

,i didnt feel humiliated;sadness
0,i can go from feeling so hopeless to so damned...
1,im grabbing a minute to post i feel greedy wro...
2,i am ever feeling nostalgic about the fireplac...
3,i am feeling grouchy;anger
4,ive been feeling a little burdened lately wasn...


In [39]:
df_val= pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/val.txt')
df_val.head()

,im feeling quite sad and sorry for myself but ill snap out of it soon;sadness
0,i feel like i am still looking at a blank canv...
1,i feel like a faithful servant;love
2,i am just feeling cranky and blue;anger
3,i can have for a treat or if i am feeling fest...
4,i start to feel more appreciative of what god ...


So we can see that the text are not properly formatted and its sucks , so we have to fix that first for 

In [40]:
df_train= pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/train.txt', delimiter=';', header=None, names=['Text','Category'])
df_test= pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/test.txt', delimiter=';', header=None, names=['Text','Category'])
df_val= pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/val.txt', delimiter=';', header=None, names=['Text','Category'])

In [41]:
df = pd.concat([df_train,df_test,df_val])
df.sample(10)

,Text,Category
11328,i can feel the longing and care and love too,love
9071,i feel im being generous with that statement,love
5732,i feel like im not as stubborn,anger
11051,i am feeling super lazy no screenshots to guid...,joy
8804,i bet taylor swift basks in the knowledge that...,fear
1099,i know their feelings are very real and not pe...,anger
4687,i feel extremely mind fucked,anger
9861,i feel like i missed the point with this book ...,sadness
285,i mainly like to text because i feel like i am...,joy
2097,i suppose most of my writing emerges out of so...,sadness


In [42]:
df['Category'].value_counts()

Category
joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: count, dtype: int64

The data is imbalace and I have to sample it so I can have a more accurate data

In [43]:
min_sample= 719

df_joy= df[df.Category=='joy'].sample(min_sample, random_state= 2345)
df_sadness= df[df.Category=='sadness'].sample(min_sample, random_state= 2345)
df_anger= df[df.Category=='anger'].sample(min_sample, random_state= 2345)
df_fear= df[df.Category=='fear'].sample(min_sample, random_state= 2345)
df_love= df[df.Category=='love'].sample(min_sample, random_state= 2345)
df_surprise= df[df.Category=='surprise'].sample(min_sample, random_state= 2345)

In [44]:
df_balanced= pd.concat([df_joy,df_sadness,df_anger,df_fear,df_love,df_surprise])
df_balanced.sample(5)

,Text,Category
7142,i feel so blessed and beyond thankful for the ...,joy
8197,i feel about it has me shocked,surprise
14600,i feel pretty rotten when jake takes off down ...,sadness
3713,i get to know about it the more guilty i feel ...,love
701,i feel assured that the guns are locked away i...,joy


In [45]:
target = {'joy':0,'sadness':1, 'anger':2,'fear':3, 'love':4,'surprise':5}
df_balanced['Category_num']=df_balanced['Category'].map(target)
df_balanced.sample(5)


,Text,Category,Category_num
7938,im sure that in a couple of months i will be f...,sadness,1
263,i would feel lucky to call any of the material...,joy,0
595,i feel like i want to hide away amp be distrac...,anger,2
9358,i supposed to feel special when you don t even...,joy,0
14077,im feeling less generous i call her psychotic,love,4


In [46]:
import spacy 
nlp= spacy.load('en_core_web_lg')

def preprocess(text):
    doc= nlp(text)
    filtered_text = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_text.append(token.lemma_)
    return " ".join(filtered_text)

In [47]:
# let me try the preprocess code and see if if working 
preprocess('The person I am destined to be is the person I have decided to be. I have to know that')

'person destine person decide know'

Now , I want to apply this to my text_column

In [48]:
df_balanced['Text_preprocessed']=df_balanced['Text'].apply(lambda Text : preprocess(Text))


In [49]:
df_balanced.sample(5)

,Text,Category,Category_num,Text_preprocessed
12562,i feel so weird about it,surprise,5,feel weird
7034,i attended a free individual academic consulta...,sadness,1,attend free individual academic consultation h...
751,i was feeling a little more resentful of what ...,anger,2,feel little resentful appear poor planning org...
4830,i don t think anyone feels curious about masal...,surprise,5,don t think feel curious masala movie light en...
9746,i feel i hate that cute patterns go out of pri...,joy,0,feel hate cute pattern print similar variation...


Now it's the time for word embeddings

In [53]:
df_balanced['embedded_text'] = df_balanced['Text'].apply(lambda x : nlp(x).vector)
df_balanced.head()

,Text,Category,Category_num,Text_preprocessed,embedded_text
5994,i feel my connection with the divine most stro...,joy,0,feel connection divine strongly feel sexy,"[-0.6760362, -0.0756277, -1.9626732, -3.363605..."
12808,i feel safe secure and protected when im in my...,joy,0,feel safe secure protect m daddys embrace,"[-0.45722693, -0.57632107, -2.9992347, -3.1258..."
14912,i was just ungrateful and selfish for wanting ...,joy,0,ungrateful selfish want life want feeling valu...,"[-2.1538985, -0.42096224, -4.34175, -1.4232463..."
6781,i i feel as though we were more successful here,joy,0,feel successful,"[-1.4471159, 0.123371996, -3.854435, -2.883726..."
1914,i actually feel halfway benevolent,joy,0,actually feel halfway benevolent,"[-0.469472, 0.048307944, -2.32918, -3.1361098,..."


In [57]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(
  df_balanced['embedded_text'].values,
    df_balanced['Category_num'],
    test_size=0.2,
    stratify= df_balanced['Category_num']   
)


In [58]:
X_train

array([array([-2.2930617e+00, -4.9222967e-01, -2.2459955e+00, -1.5473343e+00,
               9.5437771e-01, -3.0427721e+00,  2.1987343e+00,  5.8427577e+00,
              -1.0799885e-02,  5.3927773e-01,  5.2740660e+00,  2.7253292e+00,
              -2.2932045e+00, -5.8486450e-01,  1.9273124e+00,  2.5110032e-02,
               1.4804598e+00, -3.8528333e+00, -3.0827744e+00, -1.0081134e+00,
               1.9631236e+00,  2.6725209e+00, -2.4069002e+00,  5.3093994e-01,
              -1.8959833e+00, -2.6030664e+00, -3.6366644e+00, -2.7242155e+00,
               1.7938775e-01,  6.7718875e-01,  1.0140880e-01,  2.6730552e-01,
              -3.2419258e-01,  1.2619644e+00, -2.5805554e-01, -1.3912567e+00,
              -1.3686624e+00,  5.7731110e-01,  3.1041167e+00, -2.6542344e+00,
              -1.1686177e+00, -2.4167998e-01,  1.6000811e+00, -2.1682854e+00,
               2.4180443e+00,  5.7794887e-01, -1.4259477e+00, -5.6493263e+00,
              -1.5259181e+00,  3.3271444e+00,  1.8918335e-01, -2

In [59]:
#since X_test and X_Train are arrays in an array , we need to stack them into a 2D array 
import numpy as np 

stacked_X_train= np.stack(X_train)
stacked_X_test= np.stack (X_test)

In [63]:
#I also need to do the scale it becuase my machine learning models cant take in -ve value 

from sklearn.preprocessing import MinMaxScaler 

scaler= MinMaxScaler()

final_X_train= scaler.fit_tranform(stacked_X_train)
final_X_test = scaler.transform(stacked_stacked_X_test)

AttributeError: 'MinMaxScaler' object has no attribute 'fit_tranform'